In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import mediapipe as mp
from albumentations import Compose
import pandas as pd
from model import EfficientNetLandMark 
import os

In [ ]:
def extract_index_nparray(nparray):
    index = None
    for num in nparray[0]:
        index = num
        break
    return index

def calculate_delaunay_triangles(rect, points):
    """
    Calculate Delaunay triangles for a set of points.

    Args:
        rect: A tuple representing the rectangle in which to calculate the Delaunay triangles.
        points: A list of tuples representing the points.

    Returns:
        A list of tuples, each containing the indices of the 3 points forming a triangle.
    """
    # Create an instance of Subdiv2D
    subdiv = cv2.Subdiv2D(rect)

    # Insert points into subdiv
    for p in points:
        subdiv.insert((int(p[0]), int(p[1])))

    # Get Delaunay triangulation
    triangle_list = subdiv.getTriangleList()
    triangle_list = np.array(triangle_list, dtype=np.int32) 
    
    # Find the indices of triangles in the points array
    delaunay_tri = []

    for t in triangle_list:
        pt1, pt2, pt3 = (t[0], t[1]), (t[2], t[3]), (t[4], t[5])

        index_pt1 = np.where((points == pt1).all(axis=1))
        index_pt1 = extract_index_nparray(index_pt1)

        index_pt2 = np.where((points == pt2).all(axis=1))
        index_pt2 = extract_index_nparray(index_pt2)

        index_pt3 = np.where((points == pt3).all(axis=1))
        index_pt3 = extract_index_nparray(index_pt3)

        if index_pt1 is not None and index_pt2 is not None and index_pt3 is not None:
            triangle = [index_pt1, index_pt2, index_pt3]
            delaunay_tri.append(triangle)

    return delaunay_tri

def apply_affine_transform(src,src_tri,dst_tri,size):
    """
    Apply affine transform calculated using srcTri and dstTri to src and output an image of size.

    Args:
      src: Source image.
      src_tri: List of tuples representing the vertices of the source triangle.
      dst_tri: List of tuples representing the vertices of the destination triangle.
      size: A tuple representing the size of the output image.

    Returns:
      The transformed image and the warp matrix.
    """
    warp_mat = cv2.getAffineTransform(np.float32(src_tri), np.float32(dst_tri))
    dst = cv2.warpAffine(
        src, warp_mat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101
    )

    return dst

# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, t1, t2):
  # Find bounding rectangle for each triangle
  r1 = cv2.boundingRect(t1)
  r2 = cv2.boundingRect(t2)
  # Offset points by left top corner of the respective rectangles
  t1Rect = []
  t2Rect = []
  t2RectInt = []

  for i in range(0, 3):
    t1Rect.append(((t1[i][0] - r1[0]), (t1[i][1] - r1[1])))
    t2Rect.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))
    t2RectInt.append(((t2[i][0] - r2[0]), (t2[i][1] - r2[1])))

  # Get mask by filling triangle
  mask = np.zeros((r2[3], r2[2], 3), dtype=np.uint8)
  cv2.fillConvexPoly(mask, np.int32(t2RectInt), (1, 1, 1), 16, 0)

  # Apply warpImage to small rectangular patches
  img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]

  size = (r2[2], r2[3])

  img2Rect = apply_affine_transform(img1Rect, t1Rect, t2Rect, size)

  img2Rect = img2Rect * mask

  # Copy triangular region of the rectangular patch to the output image
  try:
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] *= ((1, 1, 1) - mask).astype(np.uint32)
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] += img2Rect
  except:
      pass



In [ ]:
def load_landmarks_filter(annotation_file):
        df = pd.read_csv(annotation_file, header=None)
        x = df[1].values
        y = df[2].values
        return np.array([(x1, y1) for x1, y1 in zip(x, y)])

def load_filter(filter):
    filter_path_img = '../filter/' + filter + '.png'
    filter_path_csv = '../filter/' + filter + '-annotated.csv'
    if os.path.isfile(filter_path_csv):
        pass
    else:
        filter_path_csv = '../filter/' + filter + '.csv'
    filter_ìmg = cv2.imread(filter_path_img, cv2.COLOR_BGR2RGB)
    
    filter_landmarks = load_landmarks_filter(filter_path_csv)

    points = np.array(filter_landmarks, np.int32)
    convexhull = cv2.convexHull(points)
    rect = cv2.boundingRect(convexhull)
    delau_tri_filter = calculate_delaunay_triangles(rect, points)
    
    return filter_ìmg, filter_landmarks, delau_tri_filter


In [ ]:
def get_keypoints(model, image):
    transform = Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    device = torch.device('cpu')

    input = np.array(image)
    image = Image.fromarray(image)
    input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)
    face_detector = mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.5, model_selection=1)
    faces = face_detector.process(input)
    height, width, _ = input.shape
    keypoints = []
    # plt.imshow(input)
    if not faces.detections:
        return faces, keypoints
    for face in faces.detections:
        box = face.location_data.relative_bounding_box
        x_min, y_min, x_max, y_max = box.xmin*width, box.ymin*height, (box.xmin + box.width)*width, (box.ymin + box.height)*height
        
        cropped_image = np.array(image.crop((x_min, y_min, x_max, y_max)))

        height_crop, width_crop, _ = cropped_image.shape

        detransform = Compose([
            A.Resize(width_crop, height_crop),
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

        transformed = transform(image=cropped_image)

        cropped_image = transformed["image"]
        
        input = cropped_image[None].to(device)

        output = model(input)
        points = torch.Tensor(output.cpu().reshape((68, 2))).to(torch.float32).detach().numpy()
        points = (points + 0.5) * np.array([224, 224])

        detransformed = detransform(image=np.zeros((224, 224)), keypoints=points)
        points = detransformed["keypoints"]
        points = points + np.array([x_min, y_min])
        plt.scatter(points[:, 0], points[:, 1], s=3, c='cyan')
        keypoints.append(points)

    return faces, keypoints

In [ ]:


def apply_filter_image (model, image,filter_ìmg, filter_landmarks, delau_tri_filter):

    faces, keypoints = get_keypoints(model, image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if not faces.detections:
        return image
    for i in range (len(faces.detections)):
        face_landmarks = keypoints[i]

        for tri in delau_tri_filter:
            tr1_pt1 = filter_landmarks[tri[0]]
            tr1_pt2 = filter_landmarks[tri[1]]
            tr1_pt3 = filter_landmarks[tri[2]]
            filter_tri = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)
            
            if tri[0] <= 67 and tri[1] <= 67 and tri[2] <= 67:
                tr2_pt1 = face_landmarks[tri[0]]
                tr2_pt2 = face_landmarks[tri[1]]
                tr2_pt3 = face_landmarks[tri[2]]
                face_tri = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)
                warpTriangle(filter_ìmg, image, filter_tri, face_tri)

    plt.imshow(image)
    return image
        

def apply_filter_video (model, filter):
    cap = cv2.VideoCapture(0)

    cap.set(cv2.CAP_PROP_FPS, 15)

    filter_ìmg, filter_landmarks, delau_tri_filter = load_filter(filter)
    while cap.isOpened():
        _, image = cap.read()
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = apply_filter_image(model, image, filter_ìmg, filter_landmarks, delau_tri_filter)
        image = cv2.flip(image, 1)
        cv2.imshow("Result", image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
        
        


In [ ]:
numOfPoints = 68
model = EfficientNetLandMark(numOfPoints)

model.load_state_dict(torch.load('best_model.pth', map_location=torch.device('cpu')))
device = torch.device('cpu')
print(device)
model.to(device)

image_path = '../300w/ibug/image_005_1_mirror.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
apply_filter_video(model,'anonymous')